In [ ]:
from fastai.tabular.all import *
from fastai.collab import *

import ipywidgets as widgets
from IPython.core.display import HTML, display
from ipywidgets import  interact, Layout, HBox, VBox

In [ ]:
#Obtain learner
path = Path('./movieRecBig.pkl') #0.6 civarı MSE ile bs=512 ile eğitildi.
learn = load_learner(path)

In [ ]:
#Obtain dls
dls = learn.dls

In [ ]:
#Global variables:

#General Variables
movie_factors = learn.model.i_weight.weight
maxEmbd = torch.max(movie_factors)*torch.ones([1,100])
minEmbd = torch.min(movie_factors)*torch.ones([1,100])
maxPossibleDistance = torch.dist(minEmbd,maxEmbd).item()
onceCombined = False
selectedMovies = []
combineSliders = []



#Outputs
out = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()

#HTMLs, and Labels
title =  widgets.HTML("<h2><font color='green'>Start Searching for Movies</font><h2>")
errorMsg =  widgets.HTML("<p1><font color='red'>Enter at least 3 characters.</font><p1>")
errorMsg2 =  widgets.HTML("<p1><font color='red'>Please select an item from above</font><p1>")


#Buttons
searchBtn = widgets.Button(icon="search", button_style="", layout=Layout(width='auto'))

selectBtn = widgets.Button(
    description='Add this movie',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
)

removeBtn = widgets.Button(
    description='Clear',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
)

combineBtn = widgets.Button(
    description='Combine',
    disabled=True,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
)


#TextBoxes
textBox1 = widgets.Text(
    value='',
    placeholder='Enter a movie name',
    description='Search:',
    disabled=False
)


#SelectBoxes
selectBox = widgets.Select(
    options=[''],
    value='',
    # rows=10,
    description='Movies',
    disabled=False
)

selectBox2 = widgets.Select(
    options=[''],
    value='',
    # rows=10,
    description='Selected',
    disabled=False
)

#H and V Boxes
searchHBox =  HBox([textBox1, searchBtn])
buttonGroup = HBox([combineBtn,removeBtn])
searchVBox = VBox([searchHBox,errorMsg,selectBox,errorMsg2, selectBtn])


#Layouts
errorMsg.layout.visibility = 'hidden'
errorMsg.layout.margin = "0 0 0 88px"
errorMsg2.layout.visibility = 'hidden'
errorMsg2.layout.margin = "0 0 0 88px"
selectBtn.layout.visibility = 'hidden'
selectBtn.layout.margin = "0 0 0 88px"
selectBox.layout.visibility ='hidden'
selectBox.layout.height ='200px'
selectBox.layout.width ='70%'
buttonGroup.layout.margin = "25px 0 0 87px"
out3.layout.margin = "25px 0 0 87px"

In [ ]:
def onSearchButtonPressed(a):
    selectBox.options = []
    errorMsg.layout.visibility = 'hidden'
    if(len(textBox1.value)<3):
        selectBox.layout.visibility ='hidden'
        selectBtn.layout.visibility = 'hidden'
        errorMsg.layout.visibility = 'visible'
        return
    selectBox.options = [s for s in dls.classes['title'] if textBox1.value.lower() in s.lower()] 
    ##Boş arama yapıldıysa:
    if(selectBox.options==()):
        selectBox.layout.visibility ='hidden'
        selectBtn.layout.visibility = 'hidden'
        return
            
    selectBox.layout.visibility ='visible'
    selectBtn.layout.visibility = 'visible'


def onSelectMovieButtonPressed(a):
    #if the selected movie already exists:
    if(selectBox.value in selectedMovies):
        return
    selectedMovies.append(selectBox.value)
    selectBox2.options = selectedMovies
    #selectBox2.layout.visibility ='visible'
    #btnsHBox.layout.visibility ='visible'
    createSliders(0)
    
    sum = 0
    for i,slider in enumerate(combineSliders):
        sum += slider.value
    
    if(sum==0):
        combineBtn.disabled = True
    else:
        combineBtn.disabled = False
    
def onRemoveButtonPressed(a):
    #if list is already empty:
    if(selectBox2.options==()):
        return
    selectedMovies.clear()
    selectBox2.options = selectedMovies
    
    if(onceCombined):
        createSliders(0)
    
    combineBtn.disabled = True
    out3.clear_output()
    
    
def createSliders(a):
    global onceCombined
    onceCombined = True
    combineSliders.clear()
    for movie in selectedMovies:
        combineSliders.append(createSlider(0,10))
    
    out2.clear_output()
    with out2:
        for i,slider in enumerate(combineSliders):
            movieLabel = widgets.Label(value=" "+selectedMovies[i])
            display(HBox([slider, movieLabel],layout=Layout(margin = "0 0 0 40px")))
        display(buttonGroup)


def createSlider(val, maxVal):
      
    slider = widgets.IntSlider(
        value=val,
        min=0,
        max=maxVal,
        step=1,
        readout=False
    )

    slider.observe(ratioSet, names='value')
    slider.description = "0.0%"
    
    return slider
    
def ratioSet(a): 
    
    global ratios
    ratios = []
    sum = 0
    
    #Loop through all selected movies and find the sum of slider values:
    for i,slider in enumerate(combineSliders):
        sum += slider.value
    
    if(sum==0):
        combineBtn.disabled = True
        return
    else:
        combineBtn.disabled = False
    
    #Loop through all selected movies and find the ratios for each slider:
    for i,slider in enumerate(combineSliders):
        ratios.append(slider.value/sum)
        
    for i,slider in enumerate(combineSliders):
        slider.description = str(round(100*ratios[i],1))+"%"
        
        
def Combine(a): 
        
    out3.clear_output()
    with out3:
        print("Please wait a little...")
    #Find indexes for selectedMovies
    idxs = [dls.classes['title'].o2i[m] for m in selectedMovies]

    #Calculate average embedding for selectedMovies and ratios:
    avgVec = torch.zeros([1,100])
    for i,idx in enumerate(idxs):
        rawEmbd = movie_factors[idx][None]
        ratio = ratios[i]
        embd = ratio*rawEmbd
        avgVec = torch.add(avgVec,embd)
    
    dists = torch.cdist(movie_factors,avgVec)
    distsList = dists.tolist()
    distsList = [j for sub in distsList for j in sub]
    indexedDistsList = [(idx, dist) for idx,dist in enumerate(distsList)]
    sortedDistances = sorted(indexedDistsList, key=lambda tup: tup[1])
    BOLD = '\033[1m'
    out3.clear_output()
    with out3:
        print('\033[1m'+"TOP 10 CLOSEST MOVIES TO THE COMBINATION")
        print('________________________________________\n\n')
        
        for i in range(10):
            print(str(i+1)+". "'\033[1m' + dls.classes['title'][sortedDistances[i][0]] +'\033[0m'+"  |  "+ str(round(100*(1-((sortedDistances[i][1]/maxPossibleDistance))),2)) +"% Proximity") 


In [ ]:
searchBtn.on_click(onSearchButtonPressed)
selectBtn.on_click(onSelectMovieButtonPressed)
removeBtn.on_click(onRemoveButtonPressed)
combineBtn.on_click(Combine)

## Movie Combiner App
* Movie Combiner suggests you new movies based on the selected combination of movies.
* Not sure which movie to watch? Want to watch something similar to movies you've watched before? Search and select some movies you've already watch, mix them in the ratio you want and get 10 closest movies to the combination.

## How it Works
* For example you can search for the movies **the Matrix (1999)** and **Titanic (1997)**, combine them as you wish and find the top ten closest movies to this combination.

## Under the Hood
* Movie combiner uses movie embeddings obtained by a collaborative filtering model trained on 25m movie ratings.
* By using the embedding vectors of selected movies, a new combined embedding vector is generated. After that, it is all about finding the closest L2 distances to the generated new embedding vector.

 
## Tips
* This project is done for educational purposes only, there might be some bugs on the GUI.
* Search bar only allows searches of three or more characters. So if you are looking for a movie **21** you should search as **'21 ('** or if you're looking for the movie **V** then you should search as **V (**.   

In [ ]:
#Display initial output:
out.clear_output()
with out:
    display(title)
    display(searchVBox)
out

In [ ]:
out2

In [ ]:
out3